In [1]:
import pybel
import pandas as pd
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import Rna
from pybel.dsl import MicroRna
from pybel.dsl import Fragment
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
import chembl_webresource_client
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re
from ByCovidGraph import *
import pubchempy as pcp

C:\Users\reagon.karki\Anaconda3\envs\bycovid_new\lib\site-packages\pybel\struct\operations.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
wp5 = pd.read_csv('data/normalized_data/by-covid-wp5.csv',sep=',')

In [3]:
wp5.columns

Index(['source_diagram', 'source_id', 'source_name', 'source_up',
       'source_hgnc', 'source_type', 'source_all', 'interaction_type',
       'target_id', 'target_name', 'target_up', 'target_hgnc', 'target_type',
       'target_all', 'reaction_annotation', 'org_source_id', 'org_target_id'],
      dtype='object')

In [4]:
for i in range(len(wp5)):
    if 'bind' in str(wp5['target_name'][i]):
        print(i)
        print(wp5['target_name'][i])

0
SARS-CoV-1 gRNA:RTC:nascent RNA minus strand binds RTC inhibitors
1
SARS-CoV-1 gRNA:RTC:nascent RNA minus strand binds RTC inhibitors
3
nsp13 binds nsp12
4
nsp13 binds nsp12
6
GSK3B binds GSKi
7
GSK3B binds GSKi
9
glycosylated-ACE2 binds ACE2 inhibitors
10
glycosylated-ACE2 binds ACE2 inhibitors
60
nsp15 binds RB1
61
nsp15 binds RB1
70
TMPRSS2 binds TMPRSS2 inhibitors
71
TMPRSS2 binds TMPRSS2 inhibitors
73
ZCRB1 binds 5'UTR of SARS-CoV-1 genomic RNA
74
ZCRB1 binds 5'UTR of SARS-CoV-1 genomic RNA
108
nsp8 binds MAP1LC3B
109
nsp8 binds MAP1LC3B
111
SARS-CoV-1 gRNA complement (minus strand):RTC binds RTC inhibitors
112
SARS-CoV-1 gRNA complement (minus strand):RTC binds RTC inhibitors
117
nsp14 binds nsp12
118
nsp14 binds nsp12
122
Spike glycoprotein of SARS coronavirus binds ACE2 on host cell
123
Spike glycoprotein of SARS coronavirus binds ACE2 on host cell
129
nsp14 binds nsp10
130
nsp14 binds nsp10
132
Replication transcription complex binds SARS-CoV-1 genomic RNA
133
Replication tr

In [5]:
a = 'apple'
'p' in a

True

In [6]:
test = wp5[wp5['source_up'].str.contains('UNIPROT')]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [4]:
source = wp5.loc[wp5['source_up'].str.contains('UNIPROT',na=False)]
source = source.reset_index(drop=True)
target = wp5.loc[wp5['target_up'].str.contains('UNIPROT',na=False)]
target = target.reset_index(drop=True)

In [5]:
souTar = [source,target]
souTar = pd.concat(souTar)
souTar.columns

Index(['source_diagram', 'source_id', 'source_name', 'source_up',
       'source_hgnc', 'source_type', 'source_all', 'interaction_type',
       'target_id', 'target_name', 'target_up', 'target_hgnc', 'target_type',
       'target_all', 'reaction_annotation', 'org_source_id', 'org_target_id'],
      dtype='object')

In [6]:
len(source)

2460

In [7]:
target

,source_diagram,source_id,source_name,source_up,source_hgnc,source_type,source_all,interaction_type,target_id,target_name,target_up,target_hgnc,target_type,target_all,reaction_annotation,org_source_id,org_target_id
0,R-HSA-9678108,R1_368,SARS-CoV-1 gRNA:RTC:nascent RNA minus strand b...,NaN,NaN,STATE_TRANSITION,NaN,PRODUCTION,R1_33,SARS_minus_CoV_minus_1_space_gRNA:RTC:nascent_...,UNIPROT:P0C6U8;UNIPROT:P0C6X7,NaN,COMPLEX,urn:miriam:reactome:R-COV-9694327;urn:miriam:u...,REACTOME:R-HSA-9687388;PUBMED:31645453;PUBMED:...,reaction_9687388__layout_1410,layout_1411
1,R-HSA-9678108,R1_276,nsp13 binds nsp12,NaN,NaN,STATE_TRANSITION,NaN,PRODUCTION,R1_14,nsp7:nsp8:nsp12:nsp14:nsp10:nsp13,UNIPROT:P0C6U8;UNIPROT:P0C6X7,NaN,COMPLEX,urn:miriam:uniprot:P0C6U8;urn:miriam:reactome:...,REACTOME:R-HSA-9682626;PUBMED:31131400;PUBMED:...,reaction_9682626__layout_114,layout_115
2,R-HSA-9678108,R1_370,GSK3B binds GSKi,NaN,NaN,STATE_TRANSITION,NaN,PRODUCTION,R1_67,GSK3B:GSKi,UNIPROT:P49841,NaN,COMPLEX,urn:miriam:reactome:R-HSA-9687663;urn:miriam:u...,REACTOME:R-HSA-9687724;PUBMED:34593624;PUBMED:...,reaction_9687724__layout_1893,layout_1895
3,R-HSA-9678108,R1_371,glycosylated-ACE2 binds ACE2 inhibitors,NaN,NaN,STATE_TRANSITION,NaN,PRODUCTION,R1_97,glycosylated_minus_ACE2:ACE2_space_inhibitors,UNIPROT:Q9BYF1,NaN,COMPLEX,urn:miriam:pubchem.compound:10206;urn:miriam:p...,REACTOME:R-HSA-9695415;PUBMED:32619700;PUBMED:...,reaction_9695415__layout_2063,layout_2065
4,R-HSA-9678108,R1_286,Nucleoprotein translocates to the nucleolus,NaN,NaN,STATE_TRANSITION,NaN,PRODUCTION,R1_132,SUMO_minus_p_minus_N_space_dimer,UNIPROT:P59595,NaN,COMPLEX,urn:miriam:reactome:R-COV-9683761;urn:miriam:p...,REACTOME:R-HSA-9683606;PUBMED:15848177,reaction_9683606__layout_262,layout_263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,C19DMap:Kynurenine synthesis pathway,M123_40,NaN,NaN,NaN,STATE_TRANSITION,NaN,PRODUCTION,M123_248,iNOS,UNIPROT:P35228,HGNC_SYMBOL:NOS2,PROTEIN,urn:miriam:hgnc.symbol:NOS2;urn:miriam:uniprot...,NaN,re25,sa60
1229,C19DMap:Kynurenine synthesis pathway,M123_22,NaN,NaN,NaN,STATE_TRANSITION,NaN,PRODUCTION,M123_10,Nf_minus_KB_space_Complex,UNIPROT:Q00653;UNIPROT:P19838;UNIPROT:Q04206,HGNC_SYMBOL:NFKB1;HGNC_SYMBOL:NFKB2;HGNC_SYMBO...,COMPLEX,urn:miriam:uniprot:Q00653;urn:miriam:uniprot:P...,PUBMED:15249210,re109,csa36
1230,C19DMap:Kynurenine synthesis pathway,M123_37,NaN,NaN,NaN,STATE_TRANSITION,NaN,PRODUCTION,M123_251,IDO1,UNIPROT:P14902,HGNC_SYMBOL:IDO1,RNA,urn:miriam:hgnc:6059;urn:miriam:hgnc.symbol:ID...,PUBMED:15249210;PUBMED:9126284;PUBMED:23476103,re22,sa65
1231,C19DMap:Kynurenine synthesis pathway,M123_33,NaN,NaN,NaN,HETERODIMER_ASSOCIATION,NaN,PRODUCTION,M123_4,AHR_slash_L_minus_KYN,UNIPROT:P35869,HGNC_SYMBOL:AHR,COMPLEX,urn:miriam:hgnc:348;urn:miriam:hgnc.symbol:AHR...,PUBMED:28673995;PUBMED:23123095;PUBMED:2953109...,re19,csa2


In [11]:
type(source)

pandas.core.frame.DataFrame

In [ ]:
for i in range(len(source)):
    print(source['source_up'])

In [ ]:
len(source['source_up'][3])

In [ ]:
test=source['source_up'][1].split(';')
test

In [ ]:
t = test[0].split(':')

In [ ]:
t

In [ ]:
t[1]

In [ ]:
for prot in test:
    print(prot)
    print(prot.split(':')[1])

In [8]:
wp5graph = pybel.BELGraph(name='wp5graph')

In [9]:
def wp5tobel(wp5file,itmpGraph,columnName):
    for i in range(len(wp5file)):
        prot = wp5file[columnName][i]
        print(prot)
#         print(type(prot))
        prot = prot.split(';')
        #print(prot)
#         print(prot[0].split(':'))
        for p in prot:
            print(p.split(':')[1])
            id = p.split(':')[1]
            print(wp5file['source_diagram'][i])
            itmpGraph.add_association(Protein(namespace='HP',name=id),
                                      Rna(namespace='Pathway',name=wp5file['source_diagram'][i]),
                                      citation = "WP5", evidence = 'WP5 graphs')
            
    return(itmpGraph)    
    #break
        

In [10]:
wp5_viz = wp5tobel(source,wp5graph,'source_up')
wp5_viz=wp5tobel(target,wp5graph,'target_up')

UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6X7
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P49841
P49841
R-HSA-9678108
UNIPROT:Q9BYF1
Q9BYF1
R-HSA-9678108
UNIPROT:P59595
P59595
R-HSA-9678108
UNIPROT:P59637
P59637
R-HSA-9678108
UNIPROT:P62987;UNIPROT:P62979;UNIPROT:P0CG48;UNIPROT:P0CG47
P62987
R-HSA-9678108
P62979
R-HSA-9678108
P0CG48
R-HSA-9678108
P0CG47
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X

D3DTX6
WP4861
P62136
WP4861
Q8WVI7
WP4861
Q9UQK1
WP4861
O14974
WP4861
Q96KQ4
WP4861
Q6ZSY5
WP4861
Q8TAE6
WP4861
Q96T49
WP4861
Q9H7J1
WP4861
Q9BZL4
WP4861
Q9NXH3
WP4861
Q15435
WP4861
Q9UD71
WP4861
P62140
WP4861
O95685
WP4861
Q5SRK2
WP4861
Q96C90
WP4861
B7ZBB8
WP4861
Q86XI6
WP4861
P41236
WP4861
UNIPROT:P45983
P45983
WP4861
UNIPROT:P17861
P17861
WP4861
UNIPROT:O75460
O75460
WP4861
UNIPROT:P0DTC4
P0DTC4
WP4861
UNIPROT:P17861
P17861
WP4861
UNIPROT:P17861
P17861
WP4861
UNIPROT:P11021;UNIPROT:Q9NZJ5
P11021
WP4861
Q9NZJ5
WP4861
UNIPROT:P18850
P18850
WP4861
UNIPROT:P18850
P18850
WP4861
UNIPROT:P18850
P18850
WP4861
UNIPROT:O43462;UNIPROT:Q14703
O43462
WP4861
Q14703
WP4861
UNIPROT:P17861
P17861
WP4861
UNIPROT:O75460
O75460
WP4861
UNIPROT:P0DTC2
P0DTC2
WP4861
UNIPROT:P18848
P18848
WP4861
UNIPROT:P35638
P35638
WP4861
UNIPROT:P35638
P35638
WP4861
UNIPROT:O75460
O75460
WP4861
UNIPROT:Q9NZJ5
Q9NZJ5
WP4861
UNIPROT:P31749
P31749
WP4936
UNIPROT:Q9H492
Q9H492
WP4863
UNIPROT:Q8WYN0;UNIPROT:O95352;UNIPROT:Q

Q9UHD2
C19DMap:Interferon 1 pathway
Q13114
C19DMap:Interferon 1 pathway
Q92844
C19DMap:Interferon 1 pathway
UNIPROT:P05412;UNIPROT:P01100
P05412
C19DMap:Interferon 1 pathway
P01100
C19DMap:Interferon 1 pathway
UNIPROT:O15455
O15455
C19DMap:Interferon 1 pathway
UNIPROT:O43318
O43318
C19DMap:Interferon 1 pathway
UNIPROT:Q9Y4K3
Q9Y4K3
C19DMap:Interferon 1 pathway
UNIPROT:P52630;UNIPROT:P42224;UNIPROT:Q00978
P52630
C19DMap:Interferon 1 pathway
P42224
C19DMap:Interferon 1 pathway
Q00978
C19DMap:Interferon 1 pathway
UNIPROT:Q14653
Q14653
C19DMap:Interferon 1 pathway
UNIPROT:P52630;UNIPROT:P42224;UNIPROT:Q00978
P52630
C19DMap:Interferon 1 pathway
P42224
C19DMap:Interferon 1 pathway
Q00978
C19DMap:Interferon 1 pathway
UNIPROT:Q14653
Q14653
C19DMap:Interferon 1 pathway
UNIPROT:Q9NYJ8;UNIPROT:Q9Y4K3;UNIPROT:Q15750
Q9NYJ8
C19DMap:Interferon 1 pathway
Q9Y4K3
C19DMap:Interferon 1 pathway
Q15750
C19DMap:Interferon 1 pathway
UNIPROT:P01100;UNIPROT:P05412
P01100
C19DMap:Interferon 1 pathway
P05412
C19

C19DMap:Nsp9 protein interactions
UNIPROT:P30876;UNIPROT:P24928
P30876
C19DMap:Nsp9 protein interactions
P24928
C19DMap:Nsp9 protein interactions
UNIPROT:P30876
P30876
C19DMap:Nsp9 protein interactions
UNIPROT:P19388
P19388
C19DMap:Nsp9 protein interactions
UNIPROT:P62487
P62487
C19DMap:Nsp9 protein interactions
UNIPROT:P0DTD1
P0DTD1
C19DMap:Nsp9 protein interactions
UNIPROT:Q9UBX5
Q9UBX5
C19DMap:Nsp9 protein interactions
UNIPROT:Q08397
Q08397
C19DMap:Nsp9 protein interactions
UNIPROT:P00747
P00747
C19DMap:Nsp9 protein interactions
UNIPROT:P35354
P35354
C19DMap:Nsp9 protein interactions
UNIPROT:Q96F45
Q96F45
C19DMap:Nsp9 protein interactions
UNIPROT:P61962
P61962
C19DMap:Nsp9 protein interactions
UNIPROT:P05305
P05305
C19DMap:Nsp9 protein interactions
UNIPROT:P25101
P25101
C19DMap:Nsp9 protein interactions
UNIPROT:P0DTD1
P0DTD1
C19DMap:Nsp9 protein interactions
UNIPROT:Q9HD40
Q9HD40
C19DMap:Nsp9 protein interactions
UNIPROT:Q96I59
Q96I59
C19DMap:Nsp9 protein interactions
UNIPROT:P0DTD1

P62877
C19DMap:Orf10 Cul2 pathway
Q15843
C19DMap:Orf10 Cul2 pathway
Q13617
C19DMap:Orf10 Cul2 pathway
Q15370
C19DMap:Orf10 Cul2 pathway
Q9C0D3
C19DMap:Orf10 Cul2 pathway
Q15369
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q15370;UNIPROT:Q9C0D3;UNIPROT:Q15369;UNIPROT:Q15843;UNIPROT:Q13617;UNIPROT:P62877
Q15370
C19DMap:Orf10 Cul2 pathway
Q9C0D3
C19DMap:Orf10 Cul2 pathway
Q15369
C19DMap:Orf10 Cul2 pathway
Q15843
C19DMap:Orf10 Cul2 pathway
Q13617
C19DMap:Orf10 Cul2 pathway
P62877
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q8TBC4
Q8TBC4
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q13564
Q13564
C19DMap:Orf10 Cul2 pathway
UNIPROT:P61081
P61081
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q15369;UNIPROT:Q13617;UNIPROT:Q9C0D3;UNIPROT:Q15370;UNIPROT:Q15843;UNIPROT:P62877
Q15369
C19DMap:Orf10 Cul2 pathway
Q13617
C19DMap:Orf10 Cul2 pathway
Q9C0D3
C19DMap:Orf10 Cul2 pathway
Q15370
C19DMap:Orf10 Cul2 pathway
Q15843
C19DMap:Orf10 Cul2 pathway
P62877
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q9C0D3;UNIPROT:P62877;UNIPROT:Q15370;UNIPROT:Q13617;U

P59635
R-HSA-9678108
P59637
R-HSA-9678108
P59594
R-HSA-9678108
P59596
R-HSA-9678108
UNIPROT:P59595
P59595
R-HSA-9678108
UNIPROT:P59632;UNIPROT:P59635;UNIPROT:P59637;UNIPROT:P59594;UNIPROT:P59595;UNIPROT:P59596
P59632
R-HSA-9678108
P59635
R-HSA-9678108
P59637
R-HSA-9678108
P59594
R-HSA-9678108
P59595
R-HSA-9678108
P59596
R-HSA-9678108
UNIPROT:Q9BYF1
Q9BYF1
R-HSA-9678108
UNIPROT:Q9GZQ8;UNIPROT:P0C6U8;UNIPROT:P0C6X7
Q9GZQ8
R-HSA-9678108
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P0C6X7
R-HSA-9678108
UNIPROT:P59594
P59594
R-HSA-9678108
UNIPROT:P59632;UNIPROT:P59635;UNIPROT:P59637;UNIPROT:Q9BYF1;UNIPROT:P59594;UNIPROT:P59595;UNIPROT:P59596
P59632
R-HSA-9678108
P59635
R-HSA-9678108
P59637
R-HSA-9678108
Q9BYF1
R-HSA-9678108
P59594
R-HSA-9678108
P59595
R-HSA-9678108
P59596
R-HSA-9678108
UNIPROT:P59637
P59637
R-HSA-9678108
UNIPROT:P0C6U8;UNIPROT:P0C6X7
P0C6U8
R-HSA-9678108
P

Q13114
C19DMap:Interferon 1 pathway
Q9UHD2
C19DMap:Interferon 1 pathway
Q14164
C19DMap:Interferon 1 pathway
UNIPROT:Q14164;UNIPROT:Q9UHD2;UNIPROT:Q13114;UNIPROT:Q92844
Q14164
C19DMap:Interferon 1 pathway
Q9UHD2
C19DMap:Interferon 1 pathway
Q13114
C19DMap:Interferon 1 pathway
Q92844
C19DMap:Interferon 1 pathway
UNIPROT:P05161
P05161
C19DMap:Interferon 1 pathway
UNIPROT:Q9Y6K5
Q9Y6K5
C19DMap:Interferon 1 pathway
UNIPROT:Q13114
Q13114
C19DMap:Interferon 1 pathway
UNIPROT:P01562
P01562
C19DMap:Interferon 1 pathway
UNIPROT:Q9BYX4
Q9BYX4
C19DMap:Interferon 1 pathway
UNIPROT:O15455
O15455
C19DMap:Interferon 1 pathway
UNIPROT:O95786
O95786
C19DMap:Interferon 1 pathway
UNIPROT:P45983;UNIPROT:Q16539
P45983
C19DMap:Interferon 1 pathway
Q16539
C19DMap:Interferon 1 pathway
UNIPROT:Q92985
Q92985
C19DMap:Interferon 1 pathway
UNIPROT:P52630;UNIPROT:P42224;UNIPROT:Q00978
P52630
C19DMap:Interferon 1 pathway
P42224
C19DMap:Interferon 1 pathway
Q00978
C19DMap:Interferon 1 pathway
UNIPROT:Q99836
Q99836
C19

Q9C0D3
C19DMap:Orf10 Cul2 pathway
Q15370
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q15370;UNIPROT:Q13617;UNIPROT:Q15369;UNIPROT:Q9C0D3;UNIPROT:P62877;UNIPROT:Q15843
Q15370
C19DMap:Orf10 Cul2 pathway
Q13617
C19DMap:Orf10 Cul2 pathway
Q15369
C19DMap:Orf10 Cul2 pathway
Q9C0D3
C19DMap:Orf10 Cul2 pathway
P62877
C19DMap:Orf10 Cul2 pathway
Q15843
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q15843;UNIPROT:Q13617;UNIPROT:Q9C0D3;UNIPROT:Q15369;UNIPROT:Q15370;UNIPROT:P62877
Q15843
C19DMap:Orf10 Cul2 pathway
Q13617
C19DMap:Orf10 Cul2 pathway
Q9C0D3
C19DMap:Orf10 Cul2 pathway
Q15369
C19DMap:Orf10 Cul2 pathway
Q15370
C19DMap:Orf10 Cul2 pathway
P62877
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q13617
Q13617
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q15370
Q15370
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q15369;UNIPROT:Q15370
Q15369
C19DMap:Orf10 Cul2 pathway
Q15370
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q9C0D3
Q9C0D3
C19DMap:Orf10 Cul2 pathway
UNIPROT:P62877
P62877
C19DMap:Orf10 Cul2 pathway
UNIPROT:Q15369;UNIPROT:Q13617;UNIPROT:Q15370;UNIPR

In [12]:
to_jupyter(wp5_viz)

<IPython.core.display.Javascript object>

In [11]:
wp5_viz.summarize()

---------------------  --------
Name                   wp5graph
Version
Number of Nodes        876
Number of Namespaces   2
Number of Edges        2432
Number of Annotations  0
Number of Citations    1
Number of Authors      0
Number of Components   3
Number of Warnings     0
Network Density        3.17E-03
---------------------  --------

Type (2)      Count  Example
----------  -------  ------------------------
Protein         838  p(HP:P50750)
RNA              38  r(Pathway:R-HSA-9694516)

Prefix (2)    Name                        Count  Example
------------  ------------------------  -------  ---------------------------------------
HP            Human Phenotype Ontology      838  p(HP:P38646)
Pathway                                      38  r(Pathway:"C19DMap:TGFbeta signalling")

Edge Type (1)              Count  Example
-----------------------  -------  -----------------------------------------------------------------------
RNA association Protein     1216  r(Pathway:"C19DMap:Nsp